# Making your analysis more efficient with ROOT: A basic introductory course

The ROOT dataframe tutorial shows you how to analyze datasets using `ROOT::RDataFrame`. The example analysis performs the following steps:

1. Connect a ROOT dataframe to a dataset containing 66 mio. events recorded by CMS in 2012
2. Filter the events being relevant for your analysis
3. Compute the invariant mass of the selected dimuon candidates
4. Plot the invariant mass spectrum showing resonances up to the Z mass

Specific questions, which will guide you and improve your understanding of the physics and technology, **are marked bold.**

## How to use the notebook

In short: You can execute a cell by selecting it and pressing Ctrl+Enter.

For the full documentation, you can click on `Help` above.

## Create a ROOT dataframe

The following ROOT dataframe is connected to a dataset named `Events` in a ROOT file. The file is not placed locally but pulled in via [XRootD](http://xrootd.org/) from a remote server [here](http://opendata.web.cern.ch/record/12341).

The dataset `Events` is a `TTree` (a "table" in first order) and has following branches (also refered to as "columns"):

| Branch name | Data type | Description |
|-------------|-----------|-------------|
| `nMuon` | `unsigned int` | Number of muons in this event |
| `Muon_pt` | `float[nMuon]` | Transverse momentum of the muons stored as an array of size `nMuon` |
| `Muon_eta` | `float[nMuon]` | Pseudo-rapidity of the muons stored as an array of size `nMuon` |
| `Muon_phi` | `float[nMuon]` | Azimuth of the muons stored as an array of size `nMuon` |
| `Muon_charge` | `int[nMuon]` | Charge of the muons stored as an array of size `nMuon` and either -1 or 1 |
| `Muon_mass` | `float[nMuon]` | Mass of the muons stored as an array of size `nMuon` |

In [ ]:
import ROOT
df = ROOT.RDataFrame("Events", "root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root")

## Filter relevant events for this analysis

Physics datasets are often general purpose datasets and therefore need excessive filtering of the events for the actual analysis. Here, we implement only a simple selection based on the number of muons and the charge to cut down the dataset on events, which are relevant for our study.

**Fill in the correct expressions by replacing the `<...>` parts to select ...**

1. Events with exactly two muons
2. Events with muons of opposite charge

See the table above for the column names and the data types.

In [ ]:
df_2mu = df.Filter("nMuon == <put something here>", "Events with exactly two muons")
df_os = df_2mu.Filter("Muon_charge[0] <put something here> Muon_charge[1]", "Muons with opposite charge")

## Compute the invariant mass of the dimuon system

Since we want to see the resonances in the mass spectrum, where dimuon events are more likely, we need to compute the invariant mass from the four-vectors of the muon candidates. ROOT provides a [helper](https://root.cern/doc/master/namespaceROOT_1_1VecOps.html) called `InvariantMass` which does this operation for you.

In [ ]:
df_mass = df_os.Define("Dimuon_mass", "InvariantMass(Muon_pt, Muon_eta, Muon_phi, Muon_mass)")

## Run only on a part of the dataset

The full dataset contains half a year of CMS data taking in 2012 with 66 mio events. For the purpose of this example, we use the `Range` node to run only on a small part of the dataset. This feature also comes in handy in the development phase of your analysis.

**Feel free to experiment with this parameter! How much data do you need to see all resonances from the [eta meson](https://en.wikipedia.org/wiki/Eta_meson) up to the [Z boson](https://en.wikipedia.org/wiki/W_and_Z_bosons)?**

In [ ]:
df_range = df_mass.Range(100000)

## Make a histogram of the dimuon spectrum

As (almost) always in physics, we have a look at the results in the form of a histogram. Let's book a histogram as one endpoint of our computation graph.

**Where do you expect resonances in the dimuon spectrum? Adjust the plotting range accordingly! Note that the numbers refer to GeV.**

In [ ]:
nbins = 100 # Number of bins
low = 100 # Lower edge of the histogram
up = 300 # Upper edge of the histogram
h = df_range.Histo1D(("Dimuon_mass", "Dimuon_mass", nbins, low, up), "Dimuon_mass")

## Create a plot of the dimuon spectrum

Now, the computation graph is set up. Next, we want to set up a plot.

Note that ROOT dataframe runs over the data just in the moment you need the actual result to reduce the runtime as much as possible, which is the reason this cell can take some time. The notebook magic `%%time` measures the time spend in the full cell.

In [ ]:
%%time
ROOT.gStyle.SetOptStat(0); ROOT.gStyle.SetTextFont(42)
c = ROOT.TCanvas("c", "", 800, 700)
c.SetLogx(); c.SetLogy()
h.SetTitle("")
h.GetXaxis().SetTitle("m_{#mu#mu} (GeV)"); h.GetXaxis().SetTitleSize(0.04)
h.GetYaxis().SetTitle("N_{Events}"); h.GetYaxis().SetTitleSize(0.04)
h.Draw()
TLatex label; label.SetNDC(true);
label.SetTextSize(0.040); label.DrawLatex(0.100, 0.920, "#bf{CMS Open Data}");
label.SetTextSize(0.030); label.DrawLatex(0.630, 0.920, "#sqrt{s} = 8 TeV, L_{int} = 11.6 fb^{-1}");

## Look at the plot interactively

ROOT provides for the notebooks a JavaScript front-end for drawing the canvas. Click and drag on the axis to zoom in and double click to reset view.

Don't forget that you can improve the statistics by increasing the number of events given to `Range`.

In [ ]:
%jsroot on
c->Draw()

## Inspect the cut-flow

As the last study, we have a look at the efficiency of the placed cuts.

In [ ]:
report->Print();